# **Modèle de référence (dummy)**

*Fonctionne avec Python 3.10.9 (Anaconda 23.3.1)*

## **Importation des bibliothèques**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pour les traîtements sur les variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modèle de ML utilisé
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier

# Pour gridsearch
from sklearn import model_selection

from sklearn import metrics

## **Importation du dataset**

In [2]:
dataset_a_utiliser = "data_clean.xlsx"

data = pd.read_excel(dataset_a_utiliser)

## **Fonctions**

In [3]:
def split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification):

    ##############################
    # Split du dataset en X et y #
    ##############################

    global X
    global y

    X = data.loc[:,features_list] # On ajoute nos features dans le X
    y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y

    ############################
    # Scaling et Encoding de X #
    ############################

    global feature_encoder

    numeric_transformer = StandardScaler()

    categorical_transformer = OneHotEncoder()

    feature_encoder = ColumnTransformer(
                transformers=[
                    ('cat', categorical_transformer, categorical_features),    
                    ('num', numeric_transformer, numeric_features)
                    ]
                )
    X = feature_encoder.fit_transform(X)


    ####################################
    # Split de X et y en train et test #
    ####################################

    global X_train
    global X_test
    global y_train
    global y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=0,
                                                        stratify=eval(stratification))

## **Features**

In [4]:
######################
# Choix des features #
######################

dataset_name = data

# Si rien mettre []
categorical_features = [
                        'BuildingType',
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'YearBuilt'
                       ]

# Si rien mettre []
numeric_features = [
                    'NumberofBuildings',
                    'NumberofFloors',
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)'
                   ]

# Toutes les features
features_list = categorical_features + numeric_features

## ***DummyRegressor()***

In [5]:
# Targets à estimer
targets = ['SiteEnergyUseWN_kBtu', 'TotalGHGEmissions']

# "y" pour stratifier y, sinon "None"
stratification = "None"

# Afficher les résultats détaillés (True/False)
details = False

for i in targets:

    y_column=i

    print(f"=========== [{i}] ===========\n")


    # Split & Scale du dataset
    split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification)


    # Entraînement simple
    modele = DummyRegressor() # Stratégie par défaut : mean
    modele.fit(X_train, y_train) # Étape d'entraînement
    print("--- Entraînement simple : ---")
    print(f"Le score R² du modèle pour prédire {i} est de {modele.score(X_train, y_train):.3f} (train) et {modele.score(X_test, y_test):.3f} (test).")


    ###############################################
    # Paramétrage de GridSearchCV et entraînement #
    ###############################################

    print("\n--- GridSearch : ---")

    # Fixer les valeurs des hyperparamètres à tester
    param_grid = {'strategy':['mean', 'median', 'quantile', 'constant'], 'constant':range(int(data[y_column].min()), int(data[y_column].max()), int(data[y_column].max()/3)), 'quantile':[0.0, 0.25, 0.5, 0.75, 1.0]}
    # Pour constant j'ai fait un range de min y à max y, avec un pas de max/3

    # Déterminer le score qu'on veut optimiser
    score = 'r2'

    # Je le sors ici car je vais réutiliser cette valeur plus tard
    cv = 5

    grid = model_selection.GridSearchCV(
        DummyRegressor(), # On indique le modèle à tester
        param_grid,     # hyperparamètres à tester
        cv=cv,           # nombre de folds de validation croisée
        scoring=score   # score à optimiser
    )

    # Optimiser ce modèle sur le jeu d'entraînement
    grid.fit(X_train, y_train)


    ###########################
    # Affichage des résultats #
    ###########################

    # Afficher le(s) hyperparamètre(s) optimaux
    print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement ({i}): {grid.best_params_}")

    # Afficher les performances correspondantes
    if details == True:
        print('Résultats pour chaque fold :')
        for j in range(cv):
            print(f"Fold n°{j+1} :", eval(f"grid.cv_results_['split{j}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

        print("\nRésultats de la validation croisée :")
        for mean, std, params in zip(
                grid.cv_results_['mean_test_score'], # score moyen
                grid.cv_results_['std_test_score'],  # écart-type du score
                grid.cv_results_['params']           # valeur de l'hyperparamètre
            ):

            print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")


    ###################################################################
    # Prédiction sur le jeu de test avec les hyperparamètres optimaux #
    ###################################################################

    # GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
    y_pred = grid.predict(X_test)
    print(f"Score sur le jeu de test ({i}, avec paramètres optimaux): {metrics.r2_score(y_test, y_pred):.3f}")

    print('_'*80, "\n")

    

=========== [SiteEnergyUseWN_kBtu] ===========

--- Entraînement simple : ---
Le score R² du modèle pour prédire SiteEnergyUseWN_kBtu est de 0.000 (train) et -0.013 (test).

--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (SiteEnergyUseWN_kBtu): {'constant': 58114, 'quantile': 0.75, 'strategy': 'quantile'}
Score sur le jeu de test (SiteEnergyUseWN_kBtu, avec paramètres optimaux): -0.011
________________________________________________________________________________ 

=========== [TotalGHGEmissions] ===========

--- Entraînement simple : ---
Le score R² du modèle pour prédire TotalGHGEmissions est de 0.000 (train) et -0.013 (test).

--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (TotalGHGEmissions): {'constant': 0, 'quantile': 0.75, 'strategy': 'quantile'}
Score sur le jeu de test (TotalGHGEmissions, avec paramètres optimaux): -0.013
________________________________________________________________________________ 



## ***DummyClassifier***

In [6]:
# Targets à estimer
targets = ['SiteEnergyUseWN_kBtu_cat', 'TotalGHGEmissions_cat']

# "y" pour stratifier y, sinon "None"
stratification="y"

# Afficher les résultats détaillés (True/False)
details = False

for i in targets:

    y_column=i

    print(f"=========== [{i}] ===========\n")


    # Split & Scale du dataset
    split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification)


    # Entraînement simple
    modele = DummyClassifier() # Stratégie par défaut : prior
    modele.fit(X_train, y_train) # Étape d'entraînement
    print("--- Entraînement simple : ---")
    print(f"Le score (accuracy) du modèle pour prédire {i} est de {modele.score(X_train, y_train):.3f} (train) et {modele.score(X_test, y_test):.3f} (test).")


    ###############################################
    # Paramétrage de GridSearchCV et entraînement #
    ###############################################

    print("\n--- GridSearch : ---")

    # Fixer les valeurs des hyperparamètres à tester
    param_grid = {'strategy':['most_frequent', 'prior', 'stratified', 'uniform']}
    # Pour constant j'ai fait un range de min y à max y, avec un pas de max/3

    # Déterminer le score qu'on veut optimiser
    score = 'accuracy'

    # Je le sors ici car je vais réutiliser cette valeur plus tard
    cv = 5

    grid = model_selection.GridSearchCV(
        DummyClassifier(), # On indique le modèle à tester
        param_grid,     # hyperparamètres à tester
        cv=cv,           # nombre de folds de validation croisée
        scoring=score   # score à optimiser
    )

    # Optimiser ce modèle sur le jeu d'entraînement
    grid.fit(X_train, y_train)


    ###########################
    # Affichage des résultats #
    ###########################

    # Afficher le(s) hyperparamètre(s) optimaux
    print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {grid.best_params_}")

    # Afficher les performances correspondantes
    if details == True:
        print('Résultats pour chaque fold :')
        for j in range(cv):
            print(f"Fold n°{j+1} :", eval(f"grid.cv_results_['split{j}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

        print("\nRésultats de la validation croisée :")
        for mean, std, params in zip(
                grid.cv_results_['mean_test_score'], # score moyen
                grid.cv_results_['std_test_score'],  # écart-type du score
                grid.cv_results_['params']           # valeur de l'hyperparamètre
            ):

            print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")


    ###################################################################
    # Prédiction sur le jeu de test avec les hyperparamètres optimaux #
    ###################################################################

    # GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
    y_pred = grid.predict(X_test)
    print(f"Score sur le jeu de test ({i}, avec paramètres optimaux): {metrics.accuracy_score(y_test, y_pred):.3f}")

    print('_'*80, "\n")

=========== [SiteEnergyUseWN_kBtu_cat] ===========

--- Entraînement simple : ---
Le score (accuracy) du modèle pour prédire SiteEnergyUseWN_kBtu_cat est de 0.200 (train) et 0.200 (test).

--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {'strategy': 'stratified'}
Score sur le jeu de test (SiteEnergyUseWN_kBtu_cat, avec paramètres optimaux): 0.187
________________________________________________________________________________ 

=========== [TotalGHGEmissions_cat] ===========

--- Entraînement simple : ---
Le score (accuracy) du modèle pour prédire TotalGHGEmissions_cat est de 0.200 (train) et 0.200 (test).

--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement : {'strategy': 'uniform'}
Score sur le jeu de test (TotalGHGEmissions_cat, avec paramètres optimaux): 0.180
________________________________________________________________________________ 

